wav input has been cast to required mp3 keys using the following code
  
    df.path = df.path.apply(lambda x: post_process_keys(x))

In [1]:
!cp -r ../input/python-packages2 ./

In [2]:
!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index
!tar xvfz ./python-packages2/normalizer.tgz
!pip install ./normalizer/bnunicodenormalizer-0.0.24.tar.gz -f ./ --no-index
!tar xvfz ./python-packages2/pyctcdecode.tgz
!pip install ./pyctcdecode/attrs-22.1.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/hypothesis-6.54.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pygtrie-2.5.0.tar.gz -f ./ --no-index --no-deps
!pip install ./pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps

!tar xvfz ./python-packages2/pypikenlm.tgz
!pip install ./pypikenlm/pypi-kenlm-0.1.20220713.tar.gz -f ./ --no-index --no-deps



jiwer/
jiwer/jiwer-2.3.0-py3-none-any.whl
jiwer/python-Levenshtein-0.12.2.tar.gz
jiwer/setuptools-65.3.0-py3-none-any.whl
Looking in links: ./
Processing ./jiwer/jiwer-2.3.0-py3-none-any.whl
normalizer/
normalizer/bnunicodenormalizer-0.0.24.tar.gz
Looking in links: ./
Processing ./normalizer/bnunicodenormalizer-0.0.24.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.0.24-py3-none-any.whl size=17628 sha256=4a0bd6e6b6f680838031279691509780994ebff4b6ebac62cc34f4784bf47082
  Stored in directory: /root/.cache/pip/wheels/3f/7a/fd/fde270138c2a0d8efd0e9ce20f760c2e3a0fc4317975237cb8
Successfully built bnunicodenormalizer
pyctcdecode/
pyctcdecode/hypothesis-6.54.4-py3-none-any.whl
pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl
pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl
pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl
pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
pyctcd

In [3]:
import os
import numpy as np
from tqdm.auto import tqdm
from glob import glob
from transformers import AutoFeatureExtractor, pipeline
import pandas as pd
import librosa
import IPython
from datasets import load_metric
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import gc
import wave
from scipy.io import wavfile
import scipy.signal as sps
import pyctcdecode

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")



In [4]:
# CHANGE ACCORDINGLY
BATCH_SIZE = 16
TEST_DIRECTORY = '/kaggle/input/local-stt-for-now/local_speech_for_now/sylhet'

path1 = glob(os.path.join(TEST_DIRECTORY,'*.wav'))
paths = path1

In [5]:
class CFG:
    my_model_name = '../input/yellowking-dlsprint-model/YellowKing_model'
    processor_name = '../input/yellowking-dlsprint-model/YellowKing_processor'


from transformers import Wav2Vec2ProcessorWithLM

processor = Wav2Vec2ProcessorWithLM.from_pretrained(CFG.processor_name)

In [6]:
my_asrLM = pipeline("automatic-speech-recognition", model=CFG.my_model_name ,feature_extractor =processor.feature_extractor, tokenizer= processor.tokenizer,decoder=processor.decoder ,device=0)


# speech, sr = librosa.load('/kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3', sr=processor.feature_extractor.sampling_rate)

# my_asrLM([speech]*2, chunk_length_s=112, stride_length_s=None)

# my_asrLM



class AudioDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
    def __len__(self):
        return len(self.paths)
    def __getitem__(self,idx):
        speech, sr = librosa.load(self.paths[idx], sr=processor.feature_extractor.sampling_rate) 
#         print(speech.shape)
        return speech


dataset = AudioDataset(paths)
#dataset[0]
device = 'cuda:0'

def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ])
    ## padd
    batch = [ torch.Tensor(t) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0)
    return batch, lengths, mask


dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=8, collate_fn=collate_fn_padd)



preds_all = []
for batch, lengths, mask in dataloader:
    preds = my_asrLM(list(batch.numpy().transpose()))
    preds_all+=preds



from bnunicodenormalizer import Normalizer 


bnorm = Normalizer()
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

def dari(sentence):
    try:
        if sentence[-1]!="।":
            sentence+="।"
    except:
        print(sentence)
    return sentence


df1= pd.DataFrame(
    {
        "External_ID":[p.split(os.sep)[-1].replace('.wav','') for p in paths],
        "content":[p['text']for p in preds_all]
    }
)
df1.content= df1.content.apply(lambda x:normalize(x))
df1.content= df1.content.apply(lambda x:dari(x))

In [7]:
df1

,External_ID,content
0,rec_63_audio_25,আশা করি সাকিব আমরা সব সময় জেলা কনভিন্স করিয়া স...
1,rec_66_audio_28,তারা হাসিনার বাদক ইঞ্জিনের সম্রাট গেছে গিয়া কই...
2,rec_61_audio_35,মনে স্যাম্পল কমন সেম্পল কালেকশনের পরে টেনেটেস্...
3,rec_32_audio_29,অনেক মনোপলি তাকে যাকাতদাতার বা মামরাবাজারের বা...
4,rec_26_audio_43,ওর শেষ বাসটা লাইফের স্মৃতি রক্ষা জানিল বান্ধবি...
...,...,...
3000,rec_23_audio_40,আর মা কমে কাতালানরা তার হাটিলা কিস্তি মানে খাল...
3001,rec_06_audio_76,বি তামলা করা তরফাভাবে বিজ্ঞানী বনবিদ্যা।
3002,rec_60_audio_2,ইউনিভার্সিটি আর আমরা কুদ্ধ গোষ্ঠীর পুরা সেপ্টে...
3003,rec_45_audio_11,মেডাম ভাষা বন্দিকে হবাদিকিরহবায় হলে যানাযা ছাত...


In [8]:
df1.to_csv("2.9 sylhet-yelloking-inference.csv", index=False)